# Hospital Readmission Prevention

This code was taken from an example of loan default application explained in the class of 45980 Big Data at Tepper School, Carnegie Mellon University taught by Dr. Amr Farahat. The data used for this is a cleaned version of the data available from the website - https://doi.org/10.1155/2014/781670

The cleaned and raw data are both available on the repository

The concept used in this algorithm is logistic regression and calculating the probability of readmission is explained below. 

#### Importing the Libraries

In [38]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sympy import *

#### Load the Datasets

In [3]:
train_data = pd.read_csv("Dataset/readmission_train.csv")
test_data = pd.read_csv("Dataset/readmission_test.csv")

# train and test split
y_train = train_data["readmission"]
X_train = train_data.drop("readmission", axis=1)
y_test = test_data["readmission"]
X_test = test_data.drop("readmission", axis=1)

#### Fitting a logistic regression model

In [14]:
# checking the correlation between the elements

a = pd.DataFrame(X_train.corr())
a

C:\Users\Hp\AppData\Local\Temp\ipykernel_16140\564504020.py:3: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  a = pd.DataFrame(X_train.corr())


,numberEmergency,numberInpatient,insulin,metformin,numberDiagnoses,diagAnemia,diagAsthma,diagAthlerosclerosis,diagCellulitis,diagCKD,diagDyspnea,diagHeartFailure,diagHypertension,diagHypertensiveCKD,diagPneumonia,diagSkinUlcer,timeInHospital,numNonLabProcedures,numMedications
numberEmergency,1.000000,0.263135,0.044120,-0.007924,0.054522,-0.007535,0.019523,-0.038603,0.009968,0.032091,0.007955,-0.007619,-0.025371,0.029686,-0.003759,0.004687,-0.010916,-0.035311,0.015310
numberInpatient,0.263135,1.000000,0.074493,-0.069363,0.103253,-0.010376,-0.001201,-0.056811,0.000987,0.084466,-0.024945,0.073758,-0.081380,0.098688,0.002665,0.039279,0.070520,-0.065898,0.066684
insulin,0.044120,0.074493,1.000000,-0.032876,0.077813,-0.014202,0.020576,-0.052817,0.032062,0.021988,-0.056395,0.016746,-0.071227,0.035131,0.039097,0.035985,0.097233,0.015254,0.210029
metformin,-0.007924,-0.069363,-0.032876,1.000000,-0.069568,0.010804,0.042947,-0.015875,0.036785,-0.089101,0.017816,-0.077188,0.080196,-0.108662,-0.003381,-0.001456,-0.005356,-0.042431,0.070382
numberDiagnoses,0.054522,0.103253,0.077813,-0.069568,1.000000,0.022955,-0.011619,-0.028811,0.025940,0.094173,-0.036277,0.164767,-0.254362,0.069238,0.065287,0.054470,0.219679,0.074678,0.262856
diagAnemia,-0.007535,-0.010376,-0.014202,0.010804,0.022955,1.000000,-0.020798,-0.034673,-0.024957,-0.015852,-0.028351,-0.040184,-0.033625,-0.016966,-0.027561,-0.020873,0.003619,0.055043,0.039295
diagAsthma,0.019523,-0.001201,0.020576,0.042947,-0.011619,-0.020798,1.000000,-0.032332,-0.019567,-0.021843,0.005194,-0.006992,-0.003253,-0.030641,0.035584,-0.021335,0.007312,-0.055690,0.037264
diagAthlerosclerosis,-0.038603,-0.056811,-0.052817,-0.015875,-0.028811,-0.034673,-0.032332,1.000000,-0.067614,-0.048066,0.044318,-0.014969,0.035625,-0.056549,-0.068290,-0.063368,-0.111946,0.344275,0.091699
diagCellulitis,0.009968,0.000987,0.032062,0.036785,0.025940,-0.024957,-0.019567,-0.067614,1.000000,-0.010495,-0.042650,-0.035768,-0.039195,-0.025505,-0.041301,0.191703,0.071726,-0.038095,0.010768
diagCKD,0.032091,0.084466,0.021988,-0.089101,0.094173,-0.015852,-0.021843,-0.048066,-0.010495,1.000000,-0.022176,0.023142,-0.068533,0.220902,-0.011995,-0.018233,0.000447,0.016328,0.039034


In [15]:
# finding values that are highly correlated to remove them from the model to avoid multicollinearity
min_age = 0.4
max_age = 1.0
filtered_df = a[(a >= min_age) & (a <= max_age)]
filtered_df

,numberEmergency,numberInpatient,insulin,metformin,numberDiagnoses,diagAnemia,diagAsthma,diagAthlerosclerosis,diagCellulitis,diagCKD,diagDyspnea,diagHeartFailure,diagHypertension,diagHypertensiveCKD,diagPneumonia,diagSkinUlcer,timeInHospital,numNonLabProcedures,numMedications
numberEmergency,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
numberInpatient,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
insulin,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
metformin,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
numberDiagnoses,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
diagAnemia,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
diagAsthma,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
diagAthlerosclerosis,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
diagCellulitis,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
diagCKD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [82]:
# Logistic Regression Model

# the elements timeInHospital and numMedications seem to be highly colinear, hence will be removed from the model
logreg_mod_sm = smf.logit(
    data=train_data,
    formula="readmission ~ age+numberEmergency+numberInpatient+insulin+metformin+numberDiagnoses+diagAnemia+diagAsthma+diagAthlerosclerosis+diagCellulitis+diagCKD+diagDyspnea+diagHeartFailure+diagHypertension+diagHypertensiveCKD+diagPneumonia+diagSkinUlcer+numNonLabProcedures",
)

logreg_sm = logreg_mod_sm.fit()

Optimization terminated successfully.
         Current function value: 0.337959
         Iterations 8


In [75]:
logreg_sm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:            readmission   No. Observations:                71236
Model:                          Logit   Df Residuals:                    71207
Method:                           MLE   Df Model:                           28
Date:                Mon, 10 Jul 2023   Pseudo R-squ.:                 0.03708
Time:                        09:41:28   Log-Likelihood:                -24044.
converged:                       True   LL-Null:                       -24970.
Covariance Type:            nonrobust   LLR p-value:                     0.000
========================================================================================
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept               -3.9792      0.585     -6.799      0.000      -5.126      -2.832
age[T.[10-20)]           0.6215      0.618      1.006      0.314      -0.589       1.832
age[T.[20-30)]           1.2797      0.592      2.160      0.031       0.118       2.441
age[T.[30-40)]           1.2784      0.589      2.171      0.030       0.125       2.432
age[T.[40-50)]           1.1706      0.587      1.994      0.046       0.020       2.321
age[T.[50-60)]           1.1092      0.587      1.891      0.059      -0.041       2.259
age[T.[60-70)]           1.2679      0.587      2.162      0.031       0.118       2.417
age[T.[70-80)]           1.3427      0.586      2.290      0.022       0.193       2.492
age[T.[80-90)]           1.3257      0.587      2.260      0.024       0.176       2.476
age[T.[90-100)]          1.1939      0.591      2.021      0.043       0.036       2.351
numberEmergency          0.0340      0.010      3.532      0.000       0.015       0.053
numberInpatient          0.2507      0.008     32.203      0.000       0.235       0.266
insulin                  0.0913      0.025      3.598      0.000       0.042       0.141
metformin               -0.0594      0.033     -1.822      0.068      -0.123       0.005
numberDiagnoses          0.0320      0.007      4.343      0.000       0.018       0.047
diagAnemia              -0.0883      0.073     -1.209      0.227      -0.231       0.055
diagAsthma              -0.1917      0.082     -2.340      0.019      -0.352      -0.031
diagAthlerosclerosis    -0.1179      0.043     -2.722      0.006      -0.203      -0.033
diagCellulitis          -0.1051      0.062     -1.694      0.090      -0.227       0.016
diagCKD                  0.1540      0.056      2.728      0.006       0.043       0.265
diagDyspnea             -0.2102      0.064     -3.286      0.001      -0.336      -0.085
diagHeartFailure         0.0742      0.031      2.356      0.018       0.012       0.136
diagHypertension        -0.1797      0.044     -4.070      0.000      -0.266      -0.093
diagHypertensiveCKD      0.2069      0.048      4.321      0.000       0.113       0.301
diagPneumonia           -0.2216      0.056     -3.949      0.000      -0.332      -0.112
diagSkinUlcer            0.1672      0.061      2.725      0.006       0.047       0.287
timeInHospital           0.0191      0.005      4.157      0.000       0.010       0.028
numNonLabProcedures     -0.0394      0.009     -4.532      0.000      -0.056      -0.022
numMedications           0.0087      0.002      4.654      0.000       0.005       0.012
========================================================================================
"""

From the summary, we see that the p value of age[T.[10-20)], metformin and diagAnemia are higher than 0.05, which make is non-significant values, hence the model will be re-run by eliminating these elements.

The age can't be removed as the logit function breaks the age down into different variables on it's own. This can be ignored

In [78]:
logreg_mod_sm = smf.logit(
    data=train_data,
    formula="readmission ~ age+numberEmergency+numberInpatient+insulin+numberDiagnoses+diagAsthma+diagAthlerosclerosis+diagCellulitis+diagCKD+diagDyspnea+diagHeartFailure+diagHypertension+diagHypertensiveCKD+diagPneumonia+diagSkinUlcer+numNonLabProcedures",
)
logreg_sm = logreg_mod_sm.fit()
logreg_sm.summary()

Optimization terminated successfully.
         Current function value: 0.337980
         Iterations 8


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:            readmission   No. Observations:                71236
Model:                          Logit   Df Residuals:                    71211
Method:                           MLE   Df Model:                           24
Date:                Mon, 10 Jul 2023   Pseudo R-squ.:                 0.03577
Time:                        09:41:51   Log-Likelihood:                -24076.
converged:                       True   LL-Null:                       -24970.
Covariance Type:            nonrobust   LLR p-value:                     0.000
========================================================================================
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept               -3.9397      0.585     -6.732      0.000      -5.087      -2.793
age[T.[10-20)]           0.6344      0.618      1.027      0.304      -0.576       1.845
age[T.[20-30)]           1.2882      0.592      2.174      0.030       0.127       2.449
age[T.[30-40)]           1.2970      0.589      2.203      0.028       0.143       2.451
age[T.[40-50)]           1.1978      0.587      2.041      0.041       0.047       2.348
age[T.[50-60)]           1.1409      0.587      1.945      0.052      -0.009       2.291
age[T.[60-70)]           1.3033      0.586      2.223      0.026       0.154       2.453
age[T.[70-80)]           1.3780      0.586      2.350      0.019       0.229       2.527
age[T.[80-90)]           1.3621      0.587      2.322      0.020       0.212       2.512
age[T.[90-100)]          1.2216      0.591      2.069      0.039       0.064       2.379
numberEmergency          0.0321      0.010      3.341      0.001       0.013       0.051
numberInpatient          0.2564      0.008     33.055      0.000       0.241       0.272
insulin                  0.1245      0.025      5.020      0.000       0.076       0.173
numberDiagnoses          0.0434      0.007      6.009      0.000       0.029       0.058
diagAsthma              -0.1618      0.082     -1.981      0.048      -0.322      -0.002
diagAthlerosclerosis    -0.1464      0.043     -3.431      0.001      -0.230      -0.063
diagCellulitis          -0.0860      0.062     -1.390      0.164      -0.207       0.035
diagCKD                  0.1511      0.056      2.685      0.007       0.041       0.261
diagDyspnea             -0.2526      0.064     -3.975      0.000      -0.377      -0.128
diagHeartFailure         0.0936      0.031      2.991      0.003       0.032       0.155
diagHypertension        -0.1954      0.044     -4.448      0.000      -0.282      -0.109
diagHypertensiveCKD      0.2153      0.048      4.515      0.000       0.122       0.309
diagPneumonia           -0.1822      0.056     -3.262      0.001      -0.292      -0.073
diagSkinUlcer            0.1900      0.061      3.104      0.002       0.070       0.310
numNonLabProcedures     -0.0137      0.008     -1.736      0.083      -0.029       0.002
========================================================================================
"""

To find the value of the probability of the patient being readmitted, from the details mentioned about the data, we see the following - 

1) The telehealth interventions will reduce the incidence of 30-day unplanned readmissions in the treated population by 25%
2) Cost of admission - $35000
3) Cost of admission with telehealth - $1200 + $35000

In [79]:
# Probabilities of admission

p = Symbol("p")
tele_admitted = 0.75 * (p)
tele_notadmitted = 1 - tele_admitted
admitted = p
not_admitted = 1 - admitted

# costs of admission

cost_admission = 35000
cost_telehealth = 1200
cost_noadmission = 0

# calculating the probability of admisssion -

x = (
    cost_admission * p
    + (1 - p) * cost_noadmission
    - tele_admitted * (cost_admission + cost_telehealth)
    - (1 - tele_admitted) * (cost_telehealth)
)
my_threshold = solve(x, p)
print(my_threshold)

[0.137142857142857]


In [80]:
pred_prob_logreg_sm = logreg_sm.predict(X_test)
class_logreg_sm = (pred_prob_logreg_sm > mythreshold).astype(int)
cm_logreg_sm = confusion_matrix(y_test, class_logreg_sm)

cm_logreg_sm  # The confusion matrix

array([[25971,  1175],
       [ 2943,   441]], dtype=int64)

In [56]:
TN = cm_logreg_sm[
    0, 0
]  # number of people who weren't provided with telehealth and didn't get admitted
TP = cm_logreg_sm[
    1, 1
]  # number of people who were provided with telehealth and did get admitted
FN = cm_logreg_sm[
    1, 0
]  # number of people who were provided with telehealth and didn't get admitted
FP = cm_logreg_sm[
    0, 1
]  # number of people who weren't provided with telehealth and did get admitted

In [57]:
accuracy = (cm_logreg_sm[0, 0] + cm_logreg_sm[1, 1]) / sum(sum(cm_logreg_sm))
sensitivity = (cm_logreg_sm[1, 1]) / (cm_logreg_sm[1, 0] + cm_logreg_sm[1, 1])
specificity = (cm_logreg_sm[0, 0]) / (cm_logreg_sm[0, 0] + cm_logreg_sm[0, 1])
print("Accuracy: ", round(accuracy, 2))
print("Sensitivity: ", round(sensitivity, 2))
print("Specificity: ", round(specificity, 2))

Accuracy:  0.87
Sensitivity:  0.13
Specificity:  0.96


In [58]:
x = (
    (FP * cost_telehealth)
    + (FN * cost_admission)
    + (TP * 0.75) * (cost_admission + cost_telehealth)
)  # this multiplies the
# confusion matrix elements and their respective costs. The people who were admitted into the hospital while using telehealth
# reduces by 25% as per the question. Hence, multiplied by 0.75
print("The cost of admission is $", x)
print("The cost of admission per patient is $", x / 30530)

The cost of admission is $ 116388150.0
The cost of admission per patient is $ 3812.2551588601377


In [60]:
counts = test_data[
    "readmission"
].value_counts()  # counting the people who were readmitted into the hospital
y = counts[1] * 35000  # this is the cost of admission
print("The cost of admission is $", y)
print("The cost of admission per patient is $", y / 30530)

The cost of admission is $ 118440000
The cost of admission per patient is $ 3879.462823452342


In [61]:
print("The profit of using telehealth is $", ((y - x) / 30530))

The profit of using telehealth is $ 67.2076645922044
